In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id':'11y79Tmi7znU_ancRPVVn5MWffJ1i_9G0'}) # replace the id with id of file that you want to access
downloaded.GetContentFile('data.csv')

In [0]:
import sklearn
import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn import metrics
from numpy import array
from numpy import mean
from numpy import cov
from numpy.linalg import eig
from copy import deepcopy


In [0]:
class Node:
    def __init__(self, selected_feature, split_value, depth=None, isleaf =None):
        self.selected_feature = selected_feature
        self.split_value = split_value
        self.final_label = 0
        self.left_child = None
        self.right_child = None
        self.depth = depth
        self.isleaf = 0

In [0]:
def find_max(labels):
  no_of_classes = 11 
  class_distribution = {}
  for i in range (1, no_of_classes + 1):
    class_distribution[i] = 0

  for sample in range(labels.shape[0]):
    class_distribution[labels[sample]] = class_distribution[labels[sample]] + 1

  #return max(class_distribution.items(), key=operator.itemgetter(1))[0]
  return max(class_distribution, key=class_distribution.get)

def decisionTree(data,feature, depth, max_depth):
  feature_best, number_best, gain =  Max_Split_division(data, feature)
  node = Node(feature_best, number_best)
  #node.feature_best = feature_best
 # node.number_best = number_best

  if gain == 0 or max_depth <= depth or data.shape[0] < 5:
    node1 = Node(feature_best, number_best)
    node1.isleaf = 1
    s = find_max(data[:,-1])
    node1.final_label = s
    print("node.final_label ",node.final_label )
    print(" leaf feature is ",feature_best)
    return node1

  left = []
  right = []
  for n in range(data.shape[0]):
      if data[n][feature_best] < number_best:
          left.append(data[n])
      else:
          right.append(data[n])
  left = np.asarray(left)
  right = np.asarray(right)

  node.left_child = decisionTree(left,feature,depth+1,max_depth)
  node.right_child = decisionTree(right,feature,depth+1,max_depth)

  #create node object

  return node

def populate_inital_left_right(no_of_classes):
    left_child = {}
    right_child = {}
    for i in range (1, no_of_classes + 1):
        left_child[i] = 0
        right_child[i] = 0
    return left_child, right_child

def get_distinct_features(features):
  #print("get_distinct_features",features)
  mini = np.min(features)
  maxi = np.max(features)
  range_r = maxi - mini
  std = range_r / 5
  one = mini + std
  two = one + std
  three = two + std
  four = three + std
  #five = four + std
  Lst_split_vals = [one, two, three, four]
  #print("Lst_split_vals", Lst_split_vals)
  return Lst_split_vals


#Calculated gain using gini impurity
def calculate_gain(left,right, total):
  L_sum = sum(left.values())
  R_sum = sum(right.values())
  # Calculate left and right impurites.
  if L_sum == 0:
    left_impurity = 1
  else:
    left_impurity = 1
    for key,value in left.items():
      left_impurity = left_impurity - pow((value / L_sum),2)
    
  if R_sum == 0:
    right_impurity = 1
  else:
    right_impurity = 1
    for key,value in right.items():
          right_impurity = right_impurity - pow((value / R_sum),2)
  
  total_dist = left.copy()
  total_dist.update(right)
  #print("%%%%%%%%%%%%%%%%%%%%%")
  #print(left)
  #print(right)
  #print("total_dist", total_dist)
  parent_impurity = 1
  for key,value in total_dist.items():
          parent_impurity = parent_impurity - pow((value / (L_sum + R_sum)),2)

  gain = parent_impurity - ((L_sum/ total) * left_impurity) - ((R_sum/ total) * right_impurity) 
  return gain

def partition(data, split_val, feature):  
    # CHANGE THIS
    #print(data)
    #print("data.shape[1]",data.shape[1])
    label_col = int(data.shape[1] - 1)
    #print("label_col",label_col)
    no_of_classes = 11 #len(set(data[:,label_col]))
    #label_col = 
    #print("no_of_classes",no_of_classes)
    left_child, right_child = populate_inital_left_right(no_of_classes)
    for sample in range(0, int(data.shape[0])):
        #print(data[sample][feature], split_val)
        if data[sample][feature] < split_val:
            left_child[data[sample][label_col]] =  left_child[data[sample][label_col]] + 1
        else:
            right_child[data[sample][label_col]] =  right_child[data[sample][label_col]] + 1

    return left_child, right_child

def Max_Split_division(data, features):
  max_gain = 0
 
  for feature in features:
    numbers = get_distinct_features(data[:,feature])
    print(" numbers are ", numbers)
    for number in numbers:
      left, right = partition(data, number,  feature)
      gain = calculate_gain(left,right, data.shape[0])
      if gain >= max_gain:
        global_gain = gain
        split_value = number
        selected_feature = feature
  return selected_feature, split_value, global_gain

def traverse(node):
    if node == None:
        return
    else: 
        if node.final_label != None:
            print(node.final_label, node.number_best, node.selected_feature)
        traverse(node.left_child)
        traverse(node.right_child)

def find_final_labels(X, root_node):
    print(X)
    final_labels = []
    for i in range(X.shape[0]):
        curr_node = root_node
        while(curr_node != None):
              if curr_node.isleaf == 1:
                  label = curr_node.final_label
                  print("LABEL", label)
                  final_labels.append(label)
                  break
              feature = int(curr_node.selected_feature)
              print("feature is", feature)
              value = float(X[i][feature])
              print("VAL",value)
              if  value < float(curr_node.split_value):
                  curr_node = curr_node.left_child
              else:
                  curr_node = curr_node.right_child
       
    return final_labels

def TestRandomForest(root_nodes, X_test, y_test):
    final_labels = []
    for i in range(X_test.shape[0]):
        for node in root_nodes:
          voting = []
          curr_node = node
          while(curr_node != None):
            if curr_node.selected_feature == None:
              label = curr_node.final_label
              curr_node = curr_node.left_child
            else:
              feature = int(curr_node.selected_feature)
              value = float(X_test.iloc[i][feature])
              if curr_node.split_value == None:
                  label = curr_node.final_label
                  curr_node = curr_node.left_child
              elif value < float(curr_node.split_value):
                  label = curr_node.final_label
                  curr_node = curr_node.left_child
              else:
                  label = curr_node.final_label
                  curr_node = curr_node.right_child
          voting.append(label)
        maximum = find_max(voting)
        final_labels.append(maximum)
    return final_labels



In [0]:
def RandomForest():

  dfs = pd.read_csv('data.csv',encoding='unicode_escape')
  max_class = np.max(dfs['48'])

  X1 = dfs.iloc[:,:48]
  X = X1.copy()
  for features in X1.columns:
      max_value=X1[features].max()
      min_value=X1[features].min()
      X[features]=(X1[features]-min_value)/(max_value-min_value)

  y = dfs.iloc[:,48:]
  X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)
  no_of_classes = len(y_train['48'].unique())

  global_depth = 7
  no_of_decision_trees = 4
  obj = DecisionTree()
  root_nodes = []

  for i in range(0, no_of_decision_trees):
    vertical_stack = pd.concat([X_train, y_train], axis=1)
    vertical_stack = vertical_stack.sample(frac = 0.30)
    #print(vertical_stack)
    X1 = vertical_stack.iloc[:,:48]
    y = vertical_stack.iloc[:,48:]
    #y = y.rename(columns={'48': '0'})
    #print(y)
    x =  X1[X1.columns.to_series().sample(7)]
    x_cat = pd.concat([x, y], axis=1)
    print(x_cat)
    parent_impurity = find_parent_impurity(y, no_of_classes)
    #print("parent_impurity")
    root_node = obj.generate_level(x_cat, parent_impurity, -1, global_depth, no_of_classes)
    root_nodes.append(root_node)


    print("ACCURACY of DECISON TREE :", i+ 1)
    final_labels1 = find_final_labels(X_test, root_node)
    acc1 = sklearn.metrics.accuracy_score(final_labels1, y_test.iloc[:,0].values.tolist())
    print("Decision Tree Accuracy :",acc1)


  final_labels = TestRandomForest(root_nodes, X_test, y_test)
  acc = sklearn.metrics.accuracy_score(final_labels, y_test.iloc[:,0].values.tolist())
  print("Random forest Accuracy",acc)

In [0]:
RandomForest()